In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import os
import glob
%matplotlib inline

In [2]:
train_df=pd.read_json(r'/home/yw/study/Competition/Iceberg-Classifier-Challenge_20171125/train.json')
pred_df=pd.read_json(r'/home/yw/study/Competition/Iceberg-Classifier-Challenge_20171125/test.json')

In [3]:
pred_df.head(3)

,band_1,band_2,id,inc_angle
0,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-21.629612, -21.142353, -23.908337, -28.34524...",5941774d,34.966400
1,"[-26.058969497680664, -26.058969497680664, -26...","[-25.754207611083984, -25.754207611083984, -25...",4023181e,32.615072
2,"[-14.14109992980957, -15.064241409301758, -17....","[-14.74563980102539, -14.590410232543945, -14....",b20200e4,37.505433


In [4]:
train_df.head(3)

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1


In [5]:
def generate_minibatch(inputs=None,batch_size=128,shuffle=True):
    data_cnt=inputs.shape[0]
    if batch_size>data_cnt:
        raise('batch bigger than data nmber')
    if shuffle:
        df=inputs.sample(data_cnt)
    else:
        df=inputs
    start_idx=0
    end_idx=batch_size
    while True:
        if end_idx<start_idx:
            batch_df=df.iloc[start_idx:].append(df.iloc[:end_idx])
        else:
            batch_df=df.iloc[start_idx:end_idx]
        start_idx=(start_idx+batch_size)%data_cnt
        end_idx=(start_idx+batch_size)%data_cnt
        band1=np.array([x for x in batch_df['band_1']])
        band2=np.array([x for x in batch_df['band_2']])
        imgs=np.stack([band1.reshape(-1,75,75),band2.reshape(-1,75,75)],axis=-1)
        labels=batch_df['is_iceberg'].astype(np.int32).values
        yield imgs,labels

In [6]:
mini=generate_minibatch(inputs=train_df,batch_size=128)
img,label=next(mini)

#  构建网络

In [7]:
def inference(input_tensor,keep_prob,regularizer):
    with tf.variable_scope('layer1-conv1') as scope:
        conv1_weights=tf.get_variable(name='weight',shape=[5,5,2,32],initializer=tf.random_normal_initializer(stddev=0.1,mean=0,seed=9))
        conv1_biases=tf.get_variable(name='bias',shape=[32],initializer=tf.constant_initializer(0.1))
        conv1=tf.nn.conv2d(input_tensor,conv1_weights,strides=[1,1,1,1],padding='SAME')
        relu1=tf.nn.relu(tf.nn.bias_add(conv1,conv1_biases))
        tf.summary.histogram('layer1-conv1-w',conv1_weights)
        tf.summary.histogram('layer1-conv1-b',conv1_biases)
    with tf.name_scope('layer2-pool1'):
        pool1=tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
        
    with tf.variable_scope('layer3-conv2'):
        conv2_weights=tf.get_variable(name='weight',shape=[5,5,32,64],initializer=tf.truncated_normal_initializer(stddev=0.1,mean=0.0))
        conv2_biases=tf.get_variable(name='bias',shape=64,initializer=tf.constant_initializer(0.1))
        conv2=tf.nn.conv2d(pool1,conv2_weights,strides=[1,1,1,1],padding='SAME')
        relu2=tf.nn.relu(tf.nn.bias_add(conv2,conv2_biases))
    with tf.name_scope('layer4-pool2'):
        pool2=tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
        
    with tf.variable_scope('layer5-conv3'):
        conv3_weights=tf.get_variable('weight',shape=[3,3,64,128],initializer=tf.truncated_normal_initializer(stddev=0.1,mean=0.0))
        conv3_biases=tf.get_variable('bias',shape=[128],initializer=tf.constant_initializer(0.0))
        conv3=tf.nn.conv2d(pool2,conv3_weights,strides=[1,1,1,1],padding='SAME')
        relu3=tf.nn.relu(tf.nn.bias_add(conv3,conv3_biases))
    with tf.name_scope('layer6-pool3'):
        pool3=tf.nn.max_pool(relu3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
    
    with tf.variable_scope('layer7-conv4'):
        conv4_weights=tf.get_variable(name='weight',shape=[3,3,128,128],initializer=tf.truncated_normal_initializer(stddev=0.1,mean=0.0,seed=9))
        conv4_biases=tf.get_variable(name='bias',shape=[128],initializer=tf.constant_initializer(0.1))
        conv4=tf.nn.conv2d(pool3,conv4_weights,strides=[1,1,1,1],padding='SAME')
        relu4=tf.nn.relu(tf.nn.bias_add(conv4,conv4_biases))
    with tf.name_scope('layer8-pool4'):
        pool4=tf.nn.max_pool(relu4,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
        nodes=4*4*128
        flatened=tf.reshape(pool4,[-1,nodes])
    
    with tf.variable_scope('layer9-fc1'):
        fc1_weights=tf.get_variable(name='weight',shape=[nodes,1024],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer!=None:
            tf.add_to_collection('losses',regularizer(fc1_weights))
        fc1_biases=tf.get_variable(name='bias',shape=[1024],initializer=tf.constant_initializer(0.1))
        fc1=tf.nn.relu(tf.matmul(flatened,fc1_weights)+fc1_biases)
        fc1=tf.nn.dropout(fc1,keep_prob)
    
    with tf.variable_scope('layer10-fc2'):
        fc2_weights=tf.get_variable(name='weight',shape=[1024,512],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer!=None:
            tf.add_to_collection('losses',regularizer(fc2_weights))
        fc2_biases=tf.get_variable(name='bias',shape=[512],initializer=tf.constant_initializer(0.1))
        fc2=tf.nn.relu(tf.matmul(fc1,fc2_weights)+fc2_biases)
        fc2=tf.nn.dropout(fc2,keep_prob)
    
    with tf.variable_scope('layer11-fc3'):
        fc3_weights=tf.get_variable(name='weight',shape=[512,2],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer!=None:
            tf.add_to_collection('losses',regularizer(fc3_weights))
        fc3_biases=tf.get_variable(name='bias',shape=[2],initializer=tf.constant_initializer(0.1))
        logit=tf.matmul(fc2,fc3_weights)+fc3_biases
    return logit

In [8]:
img_h=75
img_w=75
img_c=2
n_steps=1000
batch_size=64
model_path='/home/yw/study/Competition/Iceberg-Classifier-Challenge_20171125/experiments/20171125_CNN_model/cnn.ckpt'

In [9]:
conv_graph=tf.Graph()

In [10]:
with conv_graph.as_default():
    x=tf.placeholder(shape=[None,img_w,img_h,img_c],dtype=tf.float32,name='input_imgs')
    y=tf.placeholder(shape=[None],dtype=tf.int32,name='input_labels')
    keep_prob = tf.placeholder(tf.float32,name='keep_prob')
    regularizer=tf.contrib.layers.l2_regularizer(0.0001)
    logits=inference(x,keep_prob,regularizer)
    b = tf.constant(value=1,dtype=tf.float32)
    logits_eval = tf.multiply(logits,b,name='logits_eval')
    
    loss=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=y)
    avg_loss=tf.reduce_mean(loss)
    tf.summary.scalar('loss',avg_loss)
    train_op=tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    saver=tf.train.Saver()

# 训练模型

In [11]:
with tf.Session(graph=conv_graph) as sess:
    summary_merge=tf.summary.merge_all()
    writer=tf.summary.FileWriter('/home/yw/study/Competition/Iceberg-Classifier-Challenge_20171125/experiments/info_view',sess.graph)
    start_time=time.clock()
    sess.run(tf.global_variables_initializer())
    train_loss=0.0
    i=0
    for img,label in generate_minibatch(train_df,batch_size):
        _,infos=sess.run([train_op,summary_merge],feed_dict={x:img,y:label,keep_prob:0.5})
        writer.add_summary(infos,i)
        if i%100==0:
            train_loss=sess.run(avg_loss,feed_dict={x:img,y:label,keep_prob:1.0})
            print('train_loss:',train_loss)
        i+=1
        if i>n_steps:
            break
    end_time=time.clock()
    print('Time:',end_time-start_time)
    saver.save(sess,model_path)

train_loss: 4412.77
train_loss: 0.476953
train_loss: 0.51231
train_loss: 0.538695
train_loss: 0.504007
train_loss: 0.592786
train_loss: 0.565555
train_loss: 0.52059
train_loss: 0.486121
train_loss: 0.431998
train_loss: 0.410492
Time: 4997.533344


In [12]:
model_dir="/home/yw/study/Competition/Iceberg-Classifier-Challenge_20171125/experiments/20171125_CNN_model/"

In [13]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph(model_dir+"cnn.ckpt.meta")
    saver.restore(sess, model_dir+"cnn.ckpt")
    graph=tf.get_default_graph()
    x=graph.get_tensor_by_name("input_imgs:0")
    keep_prob=graph.get_tensor_by_name("keep_prob:0")
    logits=graph.get_tensor_by_name('layer11-fc3/add:0')
    pred_band1=np.array([x for x in pred_df['band_1']])
    pred_band2=np.array([x for x in pred_df['band_2']])
    pred_imgs=np.stack([pred_band1.reshape(-1,75,75),pred_band2.reshape(-1,75,75)],axis=-1)
    pred_logits=sess.run(logits,feed_dict={x:pred_imgs,keep_prob:1.0})
    print(pred_logits)

INFO:tensorflow:Restoring parameters from /home/yw/study/Competition/Iceberg-Classifier-Challenge_20171125/experiments/20171125_CNN_model/cnn.ckpt
[[  0.24976924   0.5734238 ]
 [  0.24976924   0.5734238 ]
 [ 21.60421181   4.36430502]
 ..., 
 [  0.24976924   0.5734238 ]
 [  0.05412548   1.25605106]
 [  2.05029821   0.70146734]]


In [22]:
pred_prob=(np.exp(pred_logits)/(np.exp(pred_logits).sum(axis=1)).reshape(-1,1))[:,1]

/usr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in exp
  """Entry point for launching an IPython kernel.
/usr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [24]:
pred_df['is_iceberg']=pred_prob

In [26]:
pred_df[['id','is_iceberg']].to_csv('/home/yw/study/Competition/Iceberg-Classifier-Challenge_20171125/experiments/20171125_1cnn.csv',index=False)